In [4]:
# Nhập thư viện
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import matplotlib.cm as cm
from wordcloud import WordCloud

import warnings
warnings.filterwarnings('ignore')

In [10]:
# Đọc 2 bộ dự liệu để so sánh
org_corpus = pd.read_csv('data/mental_health.csv')

cln_corpus = pd.read_csv('data/cleaned_mhc.csv')

print("Original Corpus:\n", org_corpus.head(10), "\n")
print("Cleaned Corpus:\n", cln_corpus.head(10))

print(f"\nShape of original corpus: {org_corpus.shape} and cleaned corpus: {cln_corpus.shape}")
print(f"Sample difference: {org_corpus.shape[0] - cln_corpus.shape[0]}")

Original Corpus:
                                                 text  label
0  dear american teens question dutch person hear...      0
1  nothing look forward lifei dont many reasons k...      1
2  music recommendations im looking expand playli...      0
3  im done trying feel betterthe reason im still ...      1
4  worried  year old girl subject domestic physic...      1
5  hey rredflag sure right place post this goes  ...      1
6  feel like someone needs hear tonight feeling r...      0
7  deserve liveif died right noone would carei re...      1
8  feels good ive set dateim killing friday nice ...      1
9  live guiltok made stupid random choice  its ge...      1 

Cleaned Corpus:
                                                 text  label
0  dear american teen question dutch person heard...      0
1  nothing look forward life dont many reason kee...      1
2  music recommendation im looking expand playlis...      0
3  im done trying feel reason im still alive know...      1
4  

In [7]:
# So sánh unique tokens và total tokens
def tokenize(text):
    return text.split()

def token_stats(corpus, label_column='label'):
    total_tokens_class_1 = corpus[corpus[label_column] == 1]['text'].apply(lambda x: len(tokenize(x))).sum()
    total_tokens_class_0 = corpus[corpus[label_column] == 0]['text'].apply(lambda x: len(tokenize(x))).sum()
    
    unique_tokens_class_1 = set(token for text in corpus[corpus[label_column] == 1]['text'] for token in tokenize(text))
    unique_tokens_class_0 = set(token for text in corpus[corpus[label_column] == 0]['text'] for token in tokenize(text))

    total_tokens_entire = corpus['text'].apply(lambda x: len(tokenize(x))).sum()
    unique_tokens_entire = set(token for text in corpus['text'] for token in tokenize(text))

    return {
        'total_tokens_class_1': total_tokens_class_1,
        'total_tokens_class_0': total_tokens_class_0,
        'unique_tokens_class_1': len(unique_tokens_class_1),
        'unique_tokens_class_0': len(unique_tokens_class_0),
        'total_tokens_entire': total_tokens_entire,
        'unique_tokens_entire': len(unique_tokens_entire)
    }

original_stats = token_stats(org_corpus)

cleaned_stats = token_stats(cln_corpus)

print("Original Corpus Stats:")
print(f"Class 1: Total tokens = {original_stats['total_tokens_class_1']}, Unique tokens = {original_stats['unique_tokens_class_1']}")
print(f"Class 0: Total tokens = {original_stats['total_tokens_class_0']}, Unique tokens = {original_stats['unique_tokens_class_0']}")
print(f"Entire Corpus: Total tokens = {original_stats['total_tokens_entire']}, Unique tokens = {original_stats['unique_tokens_entire']}\n")

print("Cleaned Corpus Stats:")
print(f"Class 1: Total tokens = {cleaned_stats['total_tokens_class_1']}, Unique tokens = {cleaned_stats['unique_tokens_class_1']}")
print(f"Class 0: Total tokens = {cleaned_stats['total_tokens_class_0']}, Unique tokens = {cleaned_stats['unique_tokens_class_0']}")
print(f"Entire Corpus: Total tokens = {cleaned_stats['total_tokens_entire']}, Unique tokens = {cleaned_stats['unique_tokens_entire']}")

Original Corpus Stats:
Class 1: Total tokens = 1337600, Unique tokens = 42130
Class 0: Total tokens = 670013, Unique tokens = 49022
Entire Corpus: Total tokens = 2007613, Unique tokens = 72649

Cleaned Corpus Stats:
Class 1: Total tokens = 1212239, Unique tokens = 12516
Class 0: Total tokens = 584055, Unique tokens = 13883
Entire Corpus: Total tokens = 1796294, Unique tokens = 14599


In [15]:
# So sánh sự khác nhau của phân bổ lớp
def class_distribution_with_percentage(corpus, label_column='label'):
    class_counts = corpus[label_column].value_counts()
    total_count = class_counts.sum()
    percentages = (class_counts / total_count) * 100
    return class_counts, percentages

original_counts, original_percentages = class_distribution_with_percentage(org_corpus)

cleaned_counts, cleaned_percentages = class_distribution_with_percentage(cln_corpus)

print("Original Corpus Class Distribution:")
for label in original_counts.index:
    print(f"Class {label}: Count = {original_counts[label]}, Percentage = {original_percentages[label]:.1f}%")

print("\nCleaned Corpus Class Distribution:")
for label in cleaned_counts.index:
    print(f"Class {label}: Count = {cleaned_counts[label]}, Percentage = {cleaned_percentages[label]:.1f}%")

Original Corpus Class Distribution:
Class 0: Count = 14139, Percentage = 50.5%
Class 1: Count = 13838, Percentage = 49.5%

Cleaned Corpus Class Distribution:
Class 0: Count = 14122, Percentage = 50.5%
Class 1: Count = 13818, Percentage = 49.5%


In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix

# Load the original and cleaned corpora
org_corpus = pd.read_csv('data/mental_health.csv')
cln_corpus = pd.read_csv('data/cleaned_mhc.csv')

def build_cnn_model(input_dim):
    model = keras.Sequential()
    model.add(layers.Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(input_dim, 1)))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification output
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def evaluate_cnn_tfidf(corpus):
    X = corpus['text']
    y = corpus['label']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

    # Vectorize the text data using TF-IDF
    vectorizer = TfidfVectorizer(max_features=2500)
    X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
    X_test_tfidf = vectorizer.transform(X_test).toarray()

    # Normalize the TF-IDF features
    scaler = StandardScaler()
    X_train_tfidf = scaler.fit_transform(X_train_tfidf)
    X_test_tfidf = scaler.transform(X_test_tfidf)

    # Reshape data for CNN (samples, features, channels)
    X_train_tfidf = X_train_tfidf.reshape(X_train_tfidf.shape[0], X_train_tfidf.shape[1], 1)
    X_test_tfidf = X_test_tfidf.reshape(X_test_tfidf.shape[0], X_test_tfidf.shape[1], 1)

    # Build the CNN model
    model = build_cnn_model(input_dim=X_train_tfidf.shape[1])

    # Train the model
    model.fit(X_train_tfidf, y_train, epochs=5, batch_size=32, verbose=1)

    # Make predictions
    y_pred = (model.predict(X_test_tfidf) > 0.5).astype("int32")

    # Print classification report and confusion matrix
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

# Evaluate CNN on the original corpus with TF-IDF
print("CNN on Original Corpus with TF-IDF:")
evaluate_cnn_tfidf(org_corpus)

# Evaluate CNN on the cleaned corpus with TF-IDF
print("\nCNN on Cleaned Corpus with TF-IDF:")
evaluate_cnn_tfidf(cln_corpus)

CNN on Original Corpus with TF-IDF:
Epoch 1/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 49s 109ms/step - accuracy: 0.8243 - loss: 0.3702
Epoch 2/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 45s 103ms/step - accuracy: 0.9288 - loss: 0.1807
Epoch 3/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 46s 104ms/step - accuracy: 0.9520 - loss: 0.1194
Epoch 4/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 51s 117ms/step - accuracy: 0.9739 - loss: 0.0717
Epoch 5/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 49s 111ms/step - accuracy: 0.9875 - loss: 0.0350
438/438 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.91      0.89      7126
           1       0.90      0.87      0.89      6863

    accuracy                           0.89     13989
   macro avg       0.89      0.89      0.89     13989
weighted avg       0.89      0.89      0.89     13989

Confusion Matrix:
[[6454  672]
 [ 881 5982]]

CNN on Cleaned Corpus with TF-IDF:
Epoch 1/5
437/437 ━━━━━━━━━━━━━━━━━━━━ 46s 104ms/s